In [2]:
import numpy as np
import scipy as sp
import scipy.io as sio
from scipy import ndimage
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import nipy
import nibabel #For various neuroimaging filetypes
from nibabel.analyze import load #load the image
from nipy import save_image
from nipy import load_image
import matlab.engine
import os.path

#from pCASL import mPLD_fitParkes, Parkes_fits, mPLD_getParams

%matplotlib inline
print("ab")

ab


In [4]:
f_base = '20170810_143430_ljoo_mouse_20170810_01_1_2_'
Mouse_ID = 'prac'
T2_num = '26'
CO2_num = '26'

In [5]:
CO2 = np.squeeze(np.array(nibabel.analyze.load(f_base+CO2_num+'.hdr').get_data()[:,:,:,:]))

In [6]:
dim1 = np.shape(CO2)[0]
dim2 = np.shape(CO2)[1]
nslices = 1
nframes = np.shape(CO2)[2]

### Masking data

{'CLICOLOR': '1',
 'COLORTERM': 'gnome-terminal',
 'CONDA_DEFAULT_ENV': 'py34',
 'CONDA_PATH_BACKUP': '/home16/isjoo/anaconda3/bin:/lib64/libc.so.6:/bin:/usr/bin:/usr/local/bin:/home16/isjoo:/home16/isjoo/bin:/usr/X11R6/bin:/uxapps/MATLAB/2008b/bin:/usr/local/bic/bin:/home16/isjoo/.wine/drive_c/Program:Files/Internet:Explorer/:/bin:/software/bin:/usr/bin:/usr/local/bin:/home16/isjoo:/home16/isjoo/bin:/usr/X11R6/bin:/uxapps/MATLAB/2008b/bin:/usr/local/bic/bin:/home16/isjoo/.wine/drive_c/Program:Files/Internet:Explorer/:/usr/local/abin:/usr/local/abin:/bin:/usr/bin:/usr/local/bin:/home16/isjoo:/home16/isjoo/bin:/usr/X11R6/bin:/uxapps/MATLAB/2008b/bin:/usr/local/bic/bin:/home16/isjoo/.wine/drive_c/Program:Files/Internet:Explorer/:/usr/local/abin:/usr/local/abin:/opt/Slicer3.3.6.2:/lib64/libc.so.6:/usr/lib:/usr/local/lib:/usr/X11R6/lib:/uxapps/MATLAB/2008b/bin/util/oscheck.sh:/bin:/usr/bin:/usr/local/bin:/home16/isjoo:/home16/isjoo/bin:/usr/X11R6/bin:/uxapps/MATLAB/2008b/bin:/usr/local/bic